In [ ]:
import chipwhisperer as cw
scope = cw.scope()
scope.default_setup()

Program bitfile using platform cable or other means!

Use the bitfile under `impl_hbmc`:

In [ ]:
!ls -l ../fpga/vivado/sonata.runs/impl_hbmc/sonata_top.bit

Note for the `cw.target()` call below to work, CW305.py requires an edit to block programming, in `_con()`, under the `if 'ss2' in self.platform:` block: comment out the `self.fpga.program` calls:
```python
            #if self.platform == 'ss2_ice40':
            #    self.fpga.erase_and_init()
            #    self.fpga.program(bsfile, sck_speed=prog_speed, start=True, use_fast_usb=False)
            #else:
            #    self.fpga.program(bsfile, sck_speed=prog_speed)
```

and also this:
```python
#self.fpga = CW312T_XC7A35T(scope)
```

In [ ]:
# Connect to target (but don't program it):
# warning about right number of defines is safe to ignore:
target = cw.target(scope, cw.targets.CW305, force=False, bsfile='', platform='ss2_a35', defines_files=['../fpga/hdl/sonata_defines.v'])

Diagnostic LEDs: by default, LEDs show the following:
- `USRLED0`: main clock heartbeat (25 MHz, MAINCLK on schematic)
- `USRLED1`: AES programmable clock heartbeat (MAINCLK multiplied by 4 (default) by FPGA PLL)
- `USRLED2`: HyperRAM programmable clock heartbeat (MAINCLK multiplied by 8 (default) by FPGA PLL)
- `USRLED3`: HyperRAM AXI response error
- `USRLED4`: HyperRAM controller 90 degree shifted clock is unlocked
- `USRLED5`: HyperRAM SERDES clock is unlocked
- `USRLED6`: AES programmable clock MMCM is unlocked
- `USRLED7`: HyperRAM programmable clock MMCM is unlocked
- `CHERIERR0`: SS2 error
- `CHERIERR1`: XADC error (temp/voltage out of spec)
- `CHERIERR2`: settable via register write to `target.REG_USER_LED`
- `CHERIERR3`: HyperRAM controller busy (not idle)
- all other LEDs off

Normal status is three heartbeat LEDs and no other LEDs on (except for `CHERIERR3` if a HyperRAM test is actively running).

Next we check that we can communicate with the Sonata FPGA.

**Uses SimpleSerial2 using scope's tio1/tio2 and Sonata's SER0_TX/SER0_RX (on P12 pins 2 and 4).**

**If this fails, nothing else will work!**

In [ ]:
target._ss2_test_echo()

In [ ]:
# you should also see a build time that makes sense:
target.fpga_buildtime

# LEDs and switches:

In [ ]:
# make `CHERIERR2` blink:
import time
for i in range(10):
    target.fpga_write(target.REG_USER_LED, [1])
    time.sleep(0.2)
    target.fpga_write(target.REG_USER_LED, [0])
    time.sleep(0.2)

In [ ]:
# make all the LEDs blink:
target.fpga_write(target.REG_LEDS, [0, 0, 0x60])

In [ ]:
# return LEDs to normal:
target.fpga_write(target.REG_LEDS, [0, 0, 0])

In [ ]:
# read status of input switches:
raw = int.from_bytes(target.fpga_read(target.REG_DIPS, 2), byteorder='little')
for i in range(0,16):
    if i in range(0,3): label = 'SELSW%d' % i
    elif i in range(3, 8): label = 'NAVSW%d' % (i-3)
    elif i in range(8, 16): label = 'USRSW%d' % (i-8)
    print(label, end=': ')
    if raw & 2**i:
        print('on')
    else:
        print('off')
    
    if i in [2, 7]:
        print()

This runs in an infinite loop, updating what's shown whenever a switch status changes:

In [ ]:
from IPython.display import display, clear_output

prev_raw = 0
while True:
    raw = int.from_bytes(target.fpga_read(target.REG_DIPS, 2), byteorder='little')
    if raw != prev_raw:
        prev_raw = raw
        clear_output()
        for i in range(0,16):
            if i in range(0,3): label = 'SELSW%d: ' % i
            elif i in range(3, 8): label = 'NAVSW%d: ' % (i-3)
            elif i in range(8, 16): label = 'USRSW%d: ' % (i-8)
        
            if raw & 2**i:
                label += 'on'
            else:
                label += 'off'
            display(label)

            if i in [2, 7]:
                display('')


# XADC:

In [ ]:
# connect XADC:
from chipwhisperer.capture.scopes.cwhardware.ChipWhispererHuskyMisc import XADCSettings
from chipwhisperer.common.utils import util

CODE_READ       = 0x80
CODE_WRITE      = 0xC0

class fakeoa(object):
    def __init__(self):
        pass    
    def sendMessage(self, mode, address, payload=None, Validate=False, maxResp=1, readMask=None):
        if mode != CODE_WRITE:
            return target.fpga_read(address, maxResp)
        else:
            target.fpga_write(address, payload)

oa = fakeoa()
xadc = XADCSettings(oa)
xadc.drp.data = target.REG_XADC_DRP_DATA
xadc.drp.addr = target.REG_XADC_DRP_ADDR
xadc.drp.reset_address = None

def xadc_status():
    self = xadc
    rtn = {}
    rtn['current temperature [C]'] = '%.1f' % self.temp
    rtn['maximum temperature [C]'] = '%.1f' % self.max_temp
    rtn['user temperature alarm trigger [C]'] = '%.1f' % self.temp_trigger
    rtn['user temperature reset trigger [C]'] = '%.1f' % self.temp_reset
    rtn['device temperature alarm trigger [C]'] = '%.1f' % self.ot_temp_trigger
    rtn['device temperature reset trigger [C]'] = '%.1f' % self.ot_temp_reset
    rtn['vccint'] = '%.3f' % self.vccint
    rtn['vccaux'] = '%.3f' % self.vccaux
    rtn['vccbram'] = '%.3f' % self.vccbram
    
    rtn['max vccint'] = '%.3f' % self.get_vcc('vccint', 'max')
    rtn['max vccaux'] = '%.3f' % self.get_vcc('vccaux', 'max')
    rtn['max vccbram'] = '%.3f' % self.get_vcc('vccbram', 'max')

    rtn['min vccint'] = '%.3f' % self.get_vcc('vccint', 'min')
    rtn['min vccaux'] = '%.3f' % self.get_vcc('vccaux', 'min')
    rtn['min vccbram'] = '%.3f' % self.get_vcc('vccbram', 'min')

    print(util.dict_to_str(rtn))

def xadc_clear_error():
    target.fpga_write(target.REG_XADC_STAT, [0])

We can't print the `xadc` object because it's a Husky class with more things that we have here; individual temperature and voltage measurements can be accessed as usual, or used `xadc_status()` to print everything:

In [ ]:
xadc.temp, xadc.max_temp

In [ ]:
xadc.vccint, xadc.vccaux, xadc.vccbram

In [ ]:
xadc_status()

If the temperature specified by the "user temperature alarm trigger" is exceeded, the programmable clock (which runs the AES engines and the hyperram controller) gets shut down in an effort to preserve the FPGA.

This threshold can be controlled via the `xadc.temp_trigger` property.

Regardless of this user-defined threshold, the "device temperature alarm trigger", which is an immutable build-time property (nominally 85C), also shuts down the clock.

The clock is also shutdown if any of the VCC supplies exceed their recommended operating range (you can use the `xadc._get_vcc_limit()` and `xadc._set_vcc_limit()` methods to alter these).

The clock used for the serial communication link is always kept alive, so it should (in theory!) be possible to recover from a clock shutdown without a reset.

XADC errors are sticky; run `xadc_clear_error()` to un-stick them, which should re-enable the clock (if the error condition is no longer present).

If you turn on the Sonata board's "TURBO" mode (pin 1 of SW6 to "on"), VCCINT and VCCBRAM will exceed their thresholds, so we need to increase those:

In [ ]:
xadc._set_vcc_limit(1.08, 'vccint', 'upper')
xadc._set_vcc_limit(1.08, 'vccbram', 'upper')

In [ ]:
xadc_clear_error()

## Reading ANALOGx (VAUX) signals:

In [ ]:
def xadc_analog(i=0):
    if   i == 0: j = 4
    elif i == 1: j = 12
    elif i == 2: j = 5
    elif i == 3: j = 13
    elif i == 4: j = 6
    elif i == 5: j = 14
    else: raise ValueError('Unsupported index')
    addr = 0x10 + j
    raw = xadc.drp.read(addr)
    return (raw>>4)/4096 # ref: UG480

In [ ]:
for i in range(6):
    print('ANALOG%d: %f' % (i, xadc_analog(i)))

## Reading the ANALOGx_DIGITAL signals:

In [ ]:
raw = target.fpga_read(target.REG_ANALOG_DIGITAL, 1)[0]
for i in range(0,6):
    print('ANALOG%d_DIGITAL: ' % i, end='')
    if raw & 2**i:
        print('high')
    else:
        print('low')

# Setting the programmable clocks:

There are two programmable clocks: one is used by the AES cores, the other by the HyperRAM controller.

In [ ]:
# Connect AES MMCM:
from chipwhisperer.capture.scopes.cwhardware.ChipWhispererHuskyMisc import XilinxDRP, XilinxMMCMDRP
aes_drp = XilinxDRP(oa, target.REG_MMCM_DRP_DATA, target.REG_MMCM_DRP_ADDR, target.REG_MMCM_DRP_RESET)
aes_mmcm = XilinxMMCMDRP(aes_drp)

The programmable clock can be controlled via its MMCM's divider and multiplier settings.

The input clock is 25 MHz and legal VCO range is 800 - 1600 MHz:

In [ ]:
aes_mmcm.get_main_div(), aes_mmcm.get_mul(), aes_mmcm.get_sec_div()

For example, this sets the AES clock to 25 * 50 / 11 = 113.6 MHz:

In [ ]:
aes_mmcm.set_mul(50)
aes_mmcm.set_sec_div(11)

**Be careful with these settings as it is possible to set the clock so high that it could overheat and even damage the FPGA!**

In [ ]:
# Connect HyperRAM MMCM:
hr_drp = XilinxDRP(oa, target.REG_MMCM_HR_DRP_DATA, target.REG_MMCM_HR_DRP_ADDR, target.REG_MMCM_HR_DRP_RESET)
hr_mmcm = XilinxMMCMDRP(hr_drp)

In [ ]:
hr_mmcm.get_main_div(), hr_mmcm.get_mul(), hr_mmcm.get_sec_div()

Convenience function to report current clock frequencies:

In [ ]:
def report_clocks():
    BASE_FREQ = 25
    secdiv = hr_mmcm.get_sec_div()
    maindiv = hr_mmcm.get_main_div()
    mul = hr_mmcm.get_mul()
    hr = BASE_FREQ * mul / secdiv / maindiv

    secdiv = aes_mmcm.get_sec_div()
    maindiv = aes_mmcm.get_main_div()
    mul = aes_mmcm.get_mul()
    axi = BASE_FREQ * mul / secdiv / maindiv
    
    print('HyperRAM clock: %4.1f MHz' % hr)
    print('AXI/AES  clock: %4.1f MHz' % axi)

In [ ]:
report_clocks()

# Running AES:

In [ ]:
# run AES:
# there are *ten* AES cores; each bit turns on/off the corresponding core:
CORES = 0b1111111111
target.fpga_write(target.REG_AES_CORES_EN, int.to_bytes(CORES, length=8, byteorder='little'))

# write some key and PT:
target.fpga_write(target.REG_CRYPT_KEY, range(16))
target.fpga_write(target.REG_CRYPT_TEXTIN, range(16))
target.fpga_write(target.REG_CRYPT_GO, [1])

# read result:
print(list(target.fpga_read(target.REG_CRYPT_CIPHEROUT, 16)))

To stress the board power supply and heat dissipation, run this to make it encrypt non-stop. **Keep an eye out on the XADC temperature!**

In [ ]:
# xadc.temp_trigger = 75 # optionally reduce the temp at which we shut things down

In [ ]:
# make it encrypt non-stop:
target.fpga_write(target.REG_AES_ALWAYS_ON, [1])

In [ ]:
# and stop:
target.fpga_write(target.REG_AES_ALWAYS_ON, [0])

In [ ]:
xadc_status()

If a bunch of LEDs light up and 2 of the 3 heartbeats stop, it's likely that an XADC alarm has occurred and stopped the clocks.

If it was due to high temperature, stop the AES test and/or reduce its clock, then clear the error condition; if it was due to a transient out-of-spec voltage, clearing the error condition should suffice:

In [ ]:
xadc_clear_error()

# Hyperram test:

Note this is now using the OpenHBMC HyperRAM controller. 

In [ ]:
from tqdm.notebook import tqdm
import time

def auto_hyperram_off():
    target.fpga_write(target.REG_LB_MANUAL, [3]) # turn off auto test and clear fail

def reset_hyperram():
    target.fpga_write(target.REG_HYPER_RESET, [1])
    target.fpga_write(target.REG_HYPER_RESET, [0])

def check_clocks():
    raw = target.fpga_read(target.REG_CLKSETTINGS, 1)[0]
    if not raw & 1:
        raise ValueError('AXI/AES clock is unlocked!')
    if not raw & 2:
        raise ValueError('HyperRAM clock is unlocked!')

    raw = target.fpga_read(target.REG_HYPER_STATUS, 1)[0]
    if not raw & 1:
        raise ValueError('HyperRAM 90 degree shifted clock is unlocked!')
    if not raw & 32:
        raise ValueError('HyperRAM SERDES clock is unlocked!')
        
def auto_hyperram_test(start=0, stop=8*1024*1024-4, verbose=True):
    if start % 4 or stop % 4:
        raise ValueError('Start and stop addresses need to be a multiple of 4.')
    check_clocks()
    target.fpga_write(target.REG_LB_START_ADDR, int.to_bytes(start, length=4, byteorder='little'))
    target.fpga_write(target.REG_LB_STOP_ADDR, int.to_bytes(stop,  length=4, byteorder='little'))
    target.fpga_write(target.REG_LB_MANUAL, [3]) # turn off auto test and clear fail
    config = 4 # set LFSR mode
    target.fpga_write(target.REG_LB_MANUAL, [config]) # turn on test
    if verbose: wpbar = tqdm(total=stop, desc='writes')
    prevaddr = 0
    addr = 0
    while addr < stop:
        addr = int.from_bytes(target.fpga_read(target.REG_LB_CURRENT_ADDR, 4), byteorder='little') # current address being read/written
        if addr < prevaddr:
            break
        if verbose: wpbar.update(addr - prevaddr)
        prevaddr = addr
        time.sleep(0.1)
        resp_err = target.fpga_read(target.REG_HYPER_STATUS, 1)[0] & 6
        if resp_err:
            raise ValueError("HyperRAM controller got an AXI RESP error! Reset may be required.")
    if verbose: wpbar.close()
    
    if verbose: rpbar = tqdm(total=stop, desc='reads')
    prevaddr = 0
    addr = 0
    while addr < stop:
        addr = int.from_bytes(target.fpga_read(target.REG_LB_CURRENT_ADDR, 4), byteorder='little') # current address being read/written
        if addr < prevaddr:
            break
        if verbose: rpbar.update(addr - prevaddr)
        prevaddr = addr
        time.sleep(0.1)
        resp_err = target.fpga_read(target.REG_HYPER_STATUS, 1)[0] & 6
        if resp_err:
            raise ValueError("HyperRAM controller got an AXI RESP error! Reset may be required.")

    total_errors = int.from_bytes(target.fpga_read(target.REG_LB_ERRORS, 4), byteorder='little')
    if verbose: rpbar.close()
    
    status = target.fpga_read(target.REG_HYPER_STATUS, 1)[0]

    if status & 16:
        if verbose: print('Test passed')
    else:
        print('Test FAILED; %d errors' % total_errors)
        print('Percentage of good reads: %3.2f%%' % ((1-(total_errors/((stop-start)/4)))*100))

def auto_last_address():
    return int.from_bytes(target.fpga_read(target.REG_LB_CURRENT_ADDR, 4), byteorder='little')

def auto_test_errors():
    return int.from_bytes(target.fpga_read(target.REG_LB_ERRORS, 4), byteorder='little')

def auto_hyperram_long(start=0, stop=8*1024*1024-4, iterations = 1000):
    check_clocks()
    target.fpga_write(target.REG_LB_START_ADDR, int.to_bytes(start, length=4, byteorder='little'))
    target.fpga_write(target.REG_LB_STOP_ADDR, int.to_bytes(stop,  length=4, byteorder='little')) # note AXI uses byte addressing!
    target.fpga_write(target.REG_LB_MANUAL, [3]) # turn off auto test and clear fail
    config = 4 # set LFSR mode
    target.fpga_write(target.REG_LB_MANUAL, [config]) # turn on test
    pbar = tqdm(total=iterations, desc='iterations')
    ebar = tqdm(total=1000, desc='errors')
    prev_iteration = 0
    current_iteration = 0

    prev_errors = 0
    current_errors = 0

    while current_iteration < iterations:
        current_iteration = int.from_bytes(target.fpga_read(target.REG_LB_ITERATIONS, 2), byteorder='little')
        pbar.update(current_iteration - prev_iteration)
        prev_iteration = current_iteration

        current_errors = int.from_bytes(target.fpga_read(target.REG_LB_ERRORS, 4), byteorder='little')
        ebar.update(current_errors - prev_errors)
        prev_errors = current_errors

        time.sleep(0.01)
        resp_err = target.fpga_read(target.REG_HYPER_STATUS, 1)[0] & 6
        if resp_err:
            raise ValueError("HyperRAM controller got an AXI RESP error! Reset may be required.")
    pbar.close()
    ebar.close()

    status = target.fpga_read(target.REG_HYPER_STATUS, 1)[0]

    if status & 16:
        print('Test passed')
    else:
        print('Test FAILED; %d errors' % current_errors)
        print('Percentage of good reads: %3.2f%%' % ((1-(current_errors/((stop-start)*iterations)))*100))



Automated check over the full memory space (single iteration):

In [ ]:
reset_hyperram()

In [ ]:
auto_hyperram_test()

The test keeps running after the single iteration is done:

In [ ]:
print('Last read or write address (if stuck): %d' % auto_last_address())
print('Numer of errors: %d' % auto_test_errors())
print('Number of iterations: %d' % int.from_bytes(target.fpga_read(target.REG_LB_ITERATIONS, 2), byteorder='little'))

To manually stop the test:

In [ ]:
auto_hyperram_off()

Report clock frequencies:

In [ ]:
report_clocks()

To change the clocks, set the multiplier and dividers as desired (use with care!):

In [ ]:
if False:
    # 170 MHz HyperRAM clock:
    hr_mmcm.set_mul(48)
    hr_mmcm.set_sec_div(7)

    # 150 MHz HyperRAM clock:
    hr_mmcm.set_mul(48)
    hr_mmcm.set_sec_div(8)

    # 100 MHz AXI clock:
    hr_mmcm.set_mul(48)
    hr_mmcm.set_sec_div(12)

    # 50 MHz AXI clock:
    hr_mmcm.set_mul(48)
    hr_mmcm.set_sec_div(24)

Overclocking the HyperRAM clock up to 196 MHz should work if you turn on the board's "TURBO" mode:

In [ ]:
if False:
    hr_mmcm.set_mul(55)
    hr_mmcm.set_sec_div(7)

Whenever the HyperRAM clock rate is changed, you should re-run the `reset_hyperram()` command:

In [ ]:
reset_hyperram()

In [ ]:
# can also limit the address range (e.g. for debugging); start and stop are byte addresses, so this will test addresses 0, 4, 8 and 12: 
auto_hyperram_test(start=0, stop=12)

Stress test by running multiple iterations over the full memory space (takes roughly 1 second per iteration):

In [ ]:
auto_hyperram_long(iterations=10)

Reset stress test: check that HyperRAM always works after a reset:

In [ ]:
ITERATIONS = 100
from tqdm.notebook import tnrange
for i in tnrange(ITERATIONS):
    auto_hyperram_test(verbose=False)

Python-driven memory test -- much slower; can be useful to diagnose issues.

Unlike the automated test this does not validate the full memory (it would take a few days with this approach!), however it does fully exercise the interface of the memory (with the default settings used below).

In [ ]:
def hbmc_write(address=0, wdata=0x11223344):
    check_clocks()
    target.fpga_write(target.REG_LB_MANUAL, [3]) # turn off auto test just in case
    target.fpga_write(target.REG_LB_START_ADDR, int.to_bytes(address, length=4, byteorder='little'))
    target.fpga_write(target.REG_HBMC_SINGLE_DATA, int.to_bytes(wdata, length=4, byteorder='little'))
    target.fpga_write(target.REG_HBMC_ACTION, [1])
    target.fpga_write(target.REG_HBMC_ACTION, [0])
    if not target.fpga_read(target.REG_HBMC_ACTION, 1)[0]:
        raise ValueError('HyperRAM controller is stuck!')
    return
    
def hbmc_read(address=0):
    check_clocks()
    target.fpga_write(target.REG_LB_MANUAL, [3]) # turn off auto test just in case
    target.fpga_write(target.REG_LB_START_ADDR, int.to_bytes(address, length=4, byteorder='little'))
    target.fpga_write(target.REG_HBMC_ACTION, [2])
    target.fpga_write(target.REG_HBMC_ACTION, [0])
    if not target.fpga_read(target.REG_HBMC_ACTION, 1)[0]:
        raise ValueError('HyperRAM controller is stuck!')
    return int.from_bytes(target.fpga_read(target.REG_HBMC_SINGLE_DATA, 4), byteorder='little')


In [ ]:
hbmc_write(wdata=0xffffffff)

In [ ]:
hex(hbmc_read())

In [ ]:
WORDS = 32
#WORDS = 1024

# start address for the test (divide what auto_last_address() reports by 2):
START = 0
#START = 233000//2

In [ ]:
import random
from tqdm.notebook import tnrange

target.fpga_write(target.REG_LB_MANUAL, [3]) # turn off auto test and clear fail

wdata = []
errors = 0
goods = 0
for i in tnrange(START, START+WORDS, desc='Writing'):
    data = random.randint(0, 2**32-1)
    hbmc_write(i*4, data)
    wdata.append(data)
    busy_stuck = target.fpga_read(0x83, 1)[0] & 6
    not_idle = not target.fpga_read(target.REG_HBMC_ACTION, 1)[0]
    if busy_stuck or not_idle:
        raise ValueError("Hyperram controller appears to be stuck on address %08x! Run reset_hyperram() to reset it before trying again." % (i*4))


for i in tnrange(START, START+WORDS, desc='Reading'):
    rdata = hbmc_read(i*4)
    busy_stuck = target.fpga_read(0x83, 1)[0] & 6
    not_idle = not target.fpga_read(target.REG_HBMC_ACTION, 1)[0]
    if busy_stuck or not_idle:
        raise ValueError("Hyperram controller appears to be stuck on address %08x! Re-run the reset_hyperram() and set_config() cells to reset it before trying again." % (i*2))
    if rdata != wdata[i]:
        errors += 1
        if errors < 10:
            print('i=%3d: got %08x, expected %08x, diff %08x, bits wrong: %3d' % (i, rdata, wdata[i], rdata ^ wdata[i], bin(rdata ^ wdata[i]).count('1')))
        else:
            goods += 1
    else:
        if goods == 0:
            print('Good read for i=%3d' % i)
        goods += 1

print('Percentage of good reads: %d%%' % (goods/(errors+goods)*100))
assert errors == 0, '%d errors!' % errors